# Model Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import csv
import warnings
import joblib

from pytablewriter import MarkdownTableWriter
from my_package import parse_to_dataframe
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'retina'

In [2]:
best_model = joblib.load('out/mac_batchcover.df')
adult = joblib.load('temp/adult.df')

In [3]:
batch = pd.merge(adult, best_model, how='outer')

In [4]:
batch = batch.set_index('dataset')

In [5]:
clf_list = best_model['classifier'].apply(type).unique()
clf_name = ["LogisticRegression", "RandomForestClassifier", "KNeighborsClassifier", 
            "SVC", "MLPClassifier"]
clf_clean = {clf_type : name for clf_type, name in zip(clf_list, clf_name)}

In [6]:
batch['classifier'] =  batch['classifier'].apply(lambda x: clf_clean[type(x)])
batch

,classifier__max_leaf_nodes,classifier__momentum,performance,classifier__n_estimators,roc_auc,f1_score,classifier,classifier__C,classifier__alpha,classifier__penalty,classifier__hidden_layer_sizes,runtime,acc,classifier__max_depth,classifier__max_features,classifier__n_neighbors,classifier__weights,classifier__degree,classifier__gamma
dataset,,,,,,,,,,,,,,,,,,,
adult,NaN,NaN,0.375347,NaN,0.500000,0.386500,LogisticRegression,0.1,NaN,l1,None,66.245009,0.239541,NaN,NaN,NaN,None,NaN,NaN
adult,160.0,NaN,0.704099,1024.0,0.823586,0.589534,RandomForestClassifier,NaN,NaN,None,None,790.502721,0.699176,NaN,16.0,NaN,None,NaN,NaN
adult,NaN,NaN,0.452711,NaN,0.593609,0.003629,KNeighborsClassifier,NaN,NaN,None,None,57.545938,0.760894,NaN,NaN,211.0,distance,NaN,NaN
adult,NaN,NaN,0.375347,NaN,0.500000,0.386500,SVC,0.1,NaN,None,None,1249.245182,0.239541,NaN,NaN,NaN,None,NaN,NaN
adult,NaN,0.9,0.375347,NaN,0.500000,0.386500,MLPClassifier,NaN,1.00000,None,"(4,)",995.323848,0.239541,NaN,NaN,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cover,NaN,NaN,0.854755,NaN,0.973126,0.613662,LogisticRegression,0.1,NaN,l2,None,54.061450,0.977476,NaN,NaN,NaN,None,NaN,NaN
cover,160.0,NaN,0.896815,1024.0,0.986926,0.720110,RandomForestClassifier,NaN,NaN,None,None,865.076188,0.983410,50.0,8.0,NaN,None,NaN,NaN
cover,NaN,NaN,0.844398,NaN,0.843760,0.709595,KNeighborsClassifier,NaN,NaN,None,None,308.380891,0.979839,NaN,NaN,1.0,uniform,NaN,NaN


In [23]:
by_clf = batch[['classifier', 'performance', 'roc_auc', 'f1_score', 'acc', 
       'runtime']].groupby(['classifier']).mean()
by_clf

,performance,roc_auc,f1_score,acc,runtime
classifier,,,,,
KNeighborsClassifier,0.778619,0.813102,0.608311,0.914443,114.302982
LogisticRegression,0.619846,0.696665,0.435301,0.727573,21.645752
MLPClassifier,0.800791,0.866782,0.719770,0.815819,728.927733
RandomForestClassifier,0.872586,0.947505,0.764485,0.905769,803.474464
SVC,0.792904,0.861998,0.726435,0.790278,756.426743


In [28]:
clf_by_data = batch[['classifier', 'performance', 'roc_auc', 'f1_score', 'acc', 
       'runtime']].groupby(['classifier', 'dataset']).mean()
clf_by_data.reset_index().pivot('classifier', 'dataset', 'performance')

dataset,adult,cover,letter.p1,letter.p2
classifier,,,,
KNeighborsClassifier,0.446073,0.832610,0.902240,0.933553
LogisticRegression,0.376603,0.858580,0.487622,0.756579
MLPClassifier,0.393408,0.893073,0.958818,0.957863
RandomForestClassifier,0.704364,0.897698,0.930592,0.957692
SVC,0.376695,0.878613,0.955667,0.960641
